#  Natural Language Processing Case Study

## Sentiment Classification for Customer reviews

In [1]:
! pip install wget

Defaulting to user installation because normal site-packages is not writeable
  Using cached wget-3.2.zip (10 kB)
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=11569 sha256=6ae1cfe4335f14ead48dc2e0e60f44456d8119dacb1807a554bbaaec8ef9f31f
  Stored in directory: /home/blangwallner/.cache/pip/wheels/90/1d/93/c863ee832230df5cfc25ca497b3e88e0ee3ea9e44adc46ac62
Successfully built wget
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import wget
import os

url = 'https://www.dropbox.com/s/bzvfxltd1mokp9l/amazon_reviews_train_small.csv?dl=1'
filename = wget.download(url)

## 1. Import required packages

In [3]:
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.feature_extraction import text

# import sklearn classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

## 2. Load dataset

In [5]:
data = pd.read_csv('amazon_reviews_train_small.csv')

In [6]:
data = data.sample(frac=.2)

In [7]:
data.head()

,grade,title,text
305410,1,Doesn't Like to Share,We purchased this item thinking it would be ea...
400963,2,AN OKAY BUT NOT GREAT ACTION-DISASTER FILCK,VOLCANO with TOMMY LEE JONES AND ANNE HECHE is...
372049,4,Good wheel...,Our (light weight) teddy bear hamster has used...
205099,1,InStilling Fear is more like it,I hated this book. It may be well written but ...
399411,3,Not as good as 2.0,Both of the first two CD's are in my car right...


In [8]:
data.shape

(100000, 3)

In [9]:
i = 8
print('Grade =\n{}\n'.format(data.iloc[i,0]))
print("Text = {}".format(data.iloc[i,1]))

Grade =
2

Text = Good while it lasted


##  3. Analyze Dataset

#### Check distribution of grades

In [10]:
data[['grade']].value_counts()

grade
1        20199
3        20049
4        19922
5        19919
2        19911
dtype: int64

### Nicely balanced dataset!!

### Apply tokenizing

_______________

### <span style="color:blue">**TODO: Experiment with vectorizers**</span>

* Change parameters of `CountVectorizer` and observe performance, training time etc.
* Try `TfidfVectorizer`, change parameters and observe performance, training time etc.

In [11]:
# set stopwords
stopwords = text.ENGLISH_STOP_WORDS

# initialize and fit vectorizer
vect = CountVectorizer(max_features=3000, stop_words=stopwords, token_pattern=r'\b[^\d\W]+\b')\
                      .fit(data['text'])

# apply vectorizer to data set
X = vect.transform(data['text'])

In [12]:
X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

Number of discarded tokens: since we chose to use only the most frequent tokens, the number of stop words goes up a lot!

In [13]:
print(len(vect.stop_words_))

104803


In [14]:
y = data['grade'].apply(lambda x: 1 if x > 3 else 0)

In [15]:
y

305410    0
400963    0
372049    1
205099    0
399411    0
         ..
273545    1
168030    0
22598     0
68302     1
2178      0
Name: grade, Length: 100000, dtype: int64

Basic information on tokens and size of dictionary

In [16]:
feature_names = vect.get_feature_names()

print("Number of features: {}\n".format(len(feature_names)))
print("First 100 features:\n{}\n".format(feature_names[:100]))
print("Features 110 to 130:\n{}\n".format(feature_names[110:130]))
print("Every 100th feature:\n{}\n".format(feature_names[::100]))

Number of features: 3000

First 100 features:
['ability', 'able', 'absolute', 'absolutely', 'abuse', 'ac', 'accept', 'acceptable', 'access', 'accident', 'according', 'account', 'accuracy', 'accurate', 'acoustic', 'act', 'acted', 'acting', 'action', 'actions', 'active', 'actor', 'actors', 'actress', 'acts', 'actual', 'actually', 'ad', 'adapter', 'add', 'added', 'adding', 'addition', 'additional', 'address', 'adds', 'adequate', 'adjust', 'adjustable', 'admit', 'adult', 'adults', 'advance', 'advanced', 'advantage', 'adventure', 'adventures', 'advertised', 'advertising', 'advice', 'advise', 'afraid', 'african', 'afternoon', 'age', 'ages', 'ago', 'agree', 'ahead', 'aid', 'ain', 'air', 'al', 'alarm', 'album', 'albums', 'alice', 'alien', 'alive', 'allow', 'allowed', 'allows', 'alot', 'alright', 'alternative', 'amazed', 'amazing', 'amazon', 'america', 'american', 'americans', 'amusing', 'analysis', 'ancient', 'angel', 'angle', 'angry', 'animal', 'animals', 'animation', 'anime', 'anne', 'annoye

## 4. Split into train and test set

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 5. Choose classifier method and fit on data
_______________
### <span style="color:blue">**TODO: Experiment with different classifier choices and different parameters!**</span>

Use e.g. `DecisionTreeClassifier`, `RandomForestClassifier`, `GaussianNB`, `MLPClassifier` with useful hyperparameters

In [18]:
clf = LogisticRegression(max_iter=5000)

In [19]:
#clf = clf.fit(X_train, y_train)
clf = clf.fit(X_train.toarray(), np.array(y_train))

## 6. Use trained model to predict labels for train and test set

In [20]:
y_pred = clf.predict(X_test.toarray())
y_train_pred = clf.predict(X_train.toarray())

In [21]:
test_accuracy = accuracy_score(y_test, y_pred)
train_accuracy = accuracy_score(y_train, y_train_pred)

In [22]:
print("accuracy on training set = {:1.1f}%".format(100*train_accuracy))
print("accuracy on test set\t = {:1.1f}%".format(100*test_accuracy))

accuracy on training set = 81.1%
accuracy on test set	 = 78.7%


In [23]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[10039  1988]
 [ 2274  5699]]
              precision    recall  f1-score   support

           0       0.82      0.83      0.82     12027
           1       0.74      0.71      0.73      7973

    accuracy                           0.79     20000
   macro avg       0.78      0.77      0.78     20000
weighted avg       0.79      0.79      0.79     20000



In [25]:
vect.inverse_transform(X_test.toarray()[y_test!=y_pred])[:5]

[array(['comes', 'cover', 'description', 'disappointed', 'does', 'hard',
        'kitchen', 'notice', 'page', 'print', 'read', 's', 'say', 'small',
        'white'], dtype='<U14'),
 array(['bottle', 'did', 'filler', 'fit', 'great', 'help', 'insert',
        'just', 'kit', 'lamp', 'manage', 'opening', 'perfectly', 'quality',
        'quite', 'small', 'use', 'using', 'wanted', 'wine', 'wondering'],
       dtype='<U14'),
 array(['afraid', 'auto', 'b', 'buy', 'concept', 'd', 'daily', 'did',
        'duty', 'excellent', 'expect', 'expected', 'great', 'highest',
        'house', 'light', 'odd', 'product', 'quality', 'room', 'seriously',
        'stuck', 'tool', 'use'], dtype='<U14'),
 array(['beat', 'bible', 'book', 'community', 'fish', 'good', 'isn',
        'just', 'kinda', 'know', 'like', 'ones', 'really', 'shopping',
        'similar', 't', 'tank', 'want'], dtype='<U14'),
 array(['book', 'common', 'computer', 'connect', 'details', 'equipment',
        'essential', 'experience', 'features

## Aside: Look at tokens associated with positive or negative sentiments

In [24]:
try:
    indices = np.argsort(clf.coef_[0])
    feature_names = np.array(vect.get_feature_names())[indices]
    neg_unigrams = feature_names[:100]
    print('Words associated with negative reviews:')
    print(neg_unigrams)
    pos_unigrams = feature_names[-100:-1]
    print('\n')
    print('Words associated with positive reviews:')
    print(pos_unigrams)
except:
    print('Print classifier does not have coeff attribute')

Words associated with negative reviews:
['poorly' 'waste' 'disappointing' 'disappointment' 'mediocre' 'worst'
 'worthless' 'awful' 'terrible' 'useless' 'returning' 'refund' 'junk'
 'returned' 'shallow' 'misleading' 'cheaply' 'tedious' 'horrible' 'fails'
 'lacks' 'bland' 'overpriced' 'boring' 'lacked' 'disappointed' 'poor'
 'worse' 'frustrating' 'contrived' 'lame' 'hopes' 'unfortunately'
 'unrealistic' 'ripped' 'outdated' 'okay' 'pointless' 'garbage' 'sucks'
 'lacking' 'predictable' 'dull' 'defective' 'premise' 'loses' 'barely'
 'ruined' 'disapointed' 'irritating' 'threw' 'ok' 'beware' 'wasted'
 'terribly' 'hoping' 'ridiculous' 'sorry' 'distracting' 'sadly' 'stopped'
 'dissapointed' 'weak' 'return' 'cracked' 'trash' 'failed' 'generic'
 'falls' 'repetitive' 'flimsy' 'joke' 'unless' 'needless' 'hype' 'warned'
 'pathetic' 'depressing' 'halfway' 'lousy' 'substance' 'static' 'broke'
 'rushed' 'simplistic' 'frustrated' 'suppose' 'bothered' 'dumb' 'sentence'
 'crap' 'alright' 'rip' 'whatsoever

## Apply sentiment classification to new sentence

In [25]:
sentiment = lambda x: 'positive' if x == 1 else 'negative'

### <span style="color:blue">**TODO: Write your own review and check the detected sentiment!**</span>

In [26]:
my_review = ['this product is really bad']

In [27]:
# vectorize new review
x_rev = vect.transform(my_review)
# Classify new review
y_pred = clf.predict(x_rev.toarray())
print('Sentiment of review: {}'.format(sentiment(y_pred)))

Sentiment of review: negative


## Save model to pickle files for further use

In [28]:
import pickle

In [29]:
pickle.dump(clf, open('nlp_model.pkl','wb'))

In [30]:
pickle.dump(vect, open('nlp_vect.pkl','wb'))